# Day 20: Pulse Propagation

[*Advent of Code 2023 day 20*](https://adventofcode.com/2023/day/20) and [*solution megathread*](https://redd.it/...)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2023/20/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2023%2F20%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


# %load_ext nb_mypy
# %nb_mypy On

In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

# %load_ext pycodestyle_magic
# %pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

In [4]:
part1_example_input1 = '''broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a'''


part1_example_input2 = '''broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output'''

In [5]:
from enum import Enum, auto
from dataclasses import dataclass
from typing import Dict, Tuple


class ModuleType(Enum):
    flip_flop = auto()
    conjunction = auto()
    dumb = auto()

    def __str__(self) -> str:
        match self:
            case ModuleType.flip_flop:
                return '%'
            case ModuleType.conjunction:
                return '&'
            case _:
                return ''

@dataclass
class Module:
    name: str
    type: ModuleType
    state: bool
    inputs: Dict[str, bool]
    outputs: Tuple[str]

    def __repr__(self) -> str:
        return f"{str(self.type)}{self.name} -> {', '.join(self.outputs)}"

    def register_input(self, input: str):
        self.inputs[input] = False
            
    def input_change(self, input: str, input_state: bool) -> bool:
        self.inputs[input] = input_state
        
        match self.type:
            case ModuleType.flip_flop:
                if not input_state:
                    self.state = not self.state
                    return True
            case ModuleType.conjunction:
                self.state = not all(self.inputs.values())
                return True
            case ModuleType.dumb:
                return bool(self.outputs)

In [6]:
str(ModuleType.flip_flop)

'%'

In [44]:
from typing import Iterator


def parse_input(problem_input: Iterator[str]):
    modules: Dict[str, Module] = dict()
    for line in problem_input:
        module_str, outputs_str = line.split(' -> ')
        match module_str[0]:
            case '%':
                type = ModuleType.flip_flop
                name = module_str[1:]
            case '&':
                type = ModuleType.conjunction
                name = module_str[1:]
            case _:
                type = ModuleType.dumb
                name = module_str
        modules[name] = Module(
            name=name,
            inputs=dict(),
            outputs=tuple(outputs_str.split(', ')),
            type=type,
            state=False)
    for module in list(modules.values()):
        for output_str in module.outputs:
            if output_str not in modules:
                modules[output_str] = Module(
                    name=output_str,
                    inputs=dict(),
                    outputs=tuple(),
                    type=ModuleType.dumb,
                    state=False)
            modules[output_str].register_input(module.name)
    modules['broadcaster'].register_input('button')
    return modules

modules = parse_input(part1_example_input2.splitlines())
print('\n'.join(
    module_str
    for module_str in map(str, modules.values())
    if not module_str.startswith('output')))

broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output


In [45]:
from collections import Counter

def button_signal(modules: Dict[str, Module], state: bool, debug: bool=False) -> Tuple[int, int]:
    def pulse_str(changing: str, output: str, state: bool) -> str:
        return f"{changing} -{'high' if state else 'low'}-> {output}"
    pulses = Counter()
    changing = modules['broadcaster']
    changing.input_change('button', state)
    pulses[state] += 1
    todo = [(changing, state)]
    if debug:
        print(pulse_str('button', changing.name, state))
    while todo:
        changing, state = todo.pop(0)
        for output_name in changing.outputs:
            if debug:
                print(pulse_str(changing.name, output_name, state))
            output = modules[output_name]
            if output.input_change(changing.name, state):
                pulses[output.state] += 1
                todo.append((output, output.state))
    return pulses[True], pulses[False]

pulses_high, pulses_low = 0, 0
for count in range(1,1000+1):
    p_high, p_low = button_signal(
        modules, False, debug=False)
    pulses_high += p_high
    pulses_low += p_low
    print(f"{count}: high: {pulses_high}, low: {pulses_low}")

1: high: 3, low: 3
2: high: 5, low: 5
3: high: 7, low: 9
4: high: 9, low: 11
5: high: 12, low: 14
6: high: 14, low: 16
7: high: 16, low: 20
8: high: 18, low: 22
9: high: 21, low: 25
10: high: 23, low: 27
11: high: 25, low: 31
12: high: 27, low: 33
13: high: 30, low: 36
14: high: 32, low: 38
15: high: 34, low: 42
16: high: 36, low: 44
17: high: 39, low: 47
18: high: 41, low: 49
19: high: 43, low: 53
20: high: 45, low: 55
21: high: 48, low: 58
22: high: 50, low: 60
23: high: 52, low: 64
24: high: 54, low: 66
25: high: 57, low: 69
26: high: 59, low: 71
27: high: 61, low: 75
28: high: 63, low: 77
29: high: 66, low: 80
30: high: 68, low: 82
31: high: 70, low: 86
32: high: 72, low: 88
33: high: 75, low: 91
34: high: 77, low: 93
35: high: 79, low: 97
36: high: 81, low: 99
37: high: 84, low: 102
38: high: 86, low: 104
39: high: 88, low: 108
40: high: 90, low: 110
41: high: 93, low: 113
42: high: 95, low: 115
43: high: 97, low: 119
44: high: 99, low: 121
45: high: 102, low: 124
46: high: 104, l

In [9]:
# HTML(downloaded['part2'])

In [11]:
help(Counter)

Help on class Counter in module collections:

class Counter(builtins.dict)
 |  Counter(iterable=None, /, **kwds)
 |
 |  Dict subclass for counting hashable items.  Sometimes called a bag
 |  or multiset.  Elements are stored as dictionary keys and their counts
 |  are stored as dictionary values.
 |
 |  >>> c = Counter('abcdeabcdabcaba')  # count elements from a string
 |
 |  >>> c.most_common(3)                # three most common elements
 |  [('a', 5), ('b', 4), ('c', 3)]
 |  >>> sorted(c)                       # list all unique elements
 |  ['a', 'b', 'c', 'd', 'e']
 |  >>> ''.join(sorted(c.elements()))   # list elements with repetitions
 |  'aaaaabbbbcccdde'
 |  >>> sum(c.values())                 # total of all counts
 |  15
 |
 |  >>> c['a']                          # count of letter 'a'
 |  5
 |  >>> for elem in 'shazam':           # update counts from an iterable
 |  ...     c[elem] += 1                # by adding 1 to each element's count
 |  >>> c['a']                        